In [3]:
# pip install gensim

from gensim import corpora, models, similarities
corpus = corpora.BleiCorpus('09_data/lda/ap/ap.dat', '09_data/lda/ap/vocab.txt')

In [ ]:
# data 출처 - http://www.cs.princeton.edu/~blei/lda-c/ap.tgz